Dependências

In [2]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup

Estabelecendo conexão

In [10]:
req = requests.get("https://sol.sbc.org.br/index.php/wit/issue/archive")
if  req.status_code == 200:
    print("Conexão bem sucedida!")

Conexão bem sucedida!


Obtenção das URLs de cada edição do WIT

In [11]:
soup = BeautifulSoup(req.content, 'html.parser')
#print(soup.prettify())

In [12]:
list_url = []

for a_tag in soup.find_all('a', class_='title'):
    if a_tag.get('href'):
        list_url.append(a_tag['href'])

print("URLs de Cada Edição do WIT:")
print(list_url)

URLs de Cada Edição do WIT:
['https://sol.sbc.org.br/index.php/wit/issue/view/1533', 'https://sol.sbc.org.br/index.php/wit/issue/view/1330', 'https://sol.sbc.org.br/index.php/wit/issue/view/1128', 'https://sol.sbc.org.br/index.php/wit/issue/view/956', 'https://sol.sbc.org.br/index.php/wit/issue/view/761', 'https://sol.sbc.org.br/index.php/wit/issue/view/595', 'https://sol.sbc.org.br/index.php/wit/issue/view/402', 'https://sol.sbc.org.br/index.php/wit/issue/view/210', 'https://sol.sbc.org.br/index.php/wit/issue/view/211', 'https://sol.sbc.org.br/index.php/wit/issue/view/509']


Extração dos artigos de cada edição

In [13]:
list_papers = []

for url in list_url:
    req = requests.get(url)
    if  req.status_code == 200:
        print(f"Conexão bem sucedida com a edição: {url}")

    soup = BeautifulSoup(req.content, 'html.parser')
    #print(soup.prettify())

    list_url_papers = soup.find_all('div', class_="title")

    for paper in list_url_papers:
        a_tag = paper.find('a')
        if a_tag and a_tag.get('href'):
            list_papers.append(a_tag['href'])
    


Conexão bem sucedida com a edição: https://sol.sbc.org.br/index.php/wit/issue/view/1533
Conexão bem sucedida com a edição: https://sol.sbc.org.br/index.php/wit/issue/view/1330
Conexão bem sucedida com a edição: https://sol.sbc.org.br/index.php/wit/issue/view/1128
Conexão bem sucedida com a edição: https://sol.sbc.org.br/index.php/wit/issue/view/956
Conexão bem sucedida com a edição: https://sol.sbc.org.br/index.php/wit/issue/view/761
Conexão bem sucedida com a edição: https://sol.sbc.org.br/index.php/wit/issue/view/595
Conexão bem sucedida com a edição: https://sol.sbc.org.br/index.php/wit/issue/view/402
Conexão bem sucedida com a edição: https://sol.sbc.org.br/index.php/wit/issue/view/210
Conexão bem sucedida com a edição: https://sol.sbc.org.br/index.php/wit/issue/view/211
Conexão bem sucedida com a edição: https://sol.sbc.org.br/index.php/wit/issue/view/509


In [15]:
print(list_papers)

['https://sol.sbc.org.br/index.php/wit/article/view/35966', 'https://sol.sbc.org.br/index.php/wit/article/view/35967', 'https://sol.sbc.org.br/index.php/wit/article/view/35968', 'https://sol.sbc.org.br/index.php/wit/article/view/35969', 'https://sol.sbc.org.br/index.php/wit/article/view/35970', 'https://sol.sbc.org.br/index.php/wit/article/view/35971', 'https://sol.sbc.org.br/index.php/wit/article/view/35972', 'https://sol.sbc.org.br/index.php/wit/article/view/35973', 'https://sol.sbc.org.br/index.php/wit/article/view/35974', 'https://sol.sbc.org.br/index.php/wit/article/view/35975', 'https://sol.sbc.org.br/index.php/wit/article/view/35976', 'https://sol.sbc.org.br/index.php/wit/article/view/35977', 'https://sol.sbc.org.br/index.php/wit/article/view/35978', 'https://sol.sbc.org.br/index.php/wit/article/view/35979', 'https://sol.sbc.org.br/index.php/wit/article/view/35980', 'https://sol.sbc.org.br/index.php/wit/article/view/35981', 'https://sol.sbc.org.br/index.php/wit/article/view/3598

In [16]:
print(len(list_papers))

421


Extração dos metadados de cada artigo

In [ ]:
dados_artigos = []

for paper in list_papers:
    req = requests.get(paper)
    if req.status_code != 200:
        continue

    print(f"Conexão bem sucedida com o paper: {paper}")
    soup = BeautifulSoup(req.content, 'html.parser')

    # Edição (robusta)
    edicao = None
    for a in soup.select('nav.cmp_breadcrumbs a[href]'):
        if '/issue/view/' in a['href']:
            edicao = a.get_text(strip=True)
            break

    # Categoria
    categoria_tag = soup.select_one('nav.cmp_breadcrumbs li.current')
    categoria = categoria_tag.get_text(strip=True) if categoria_tag else None

    # Título
    titulo_tag = soup.find('h1', class_='page_title')
    titulo = titulo_tag.get_text(strip=True) if titulo_tag else None

    # Autores
    autores = []
    for li in soup.select('ul.item.authors li'):
        nome = li.select_one('span.name')
        afil = li.select_one('span.affiliation')
        if nome and afil:
            autores.append(f"{nome.get_text(strip=True)} ({afil.get_text(strip=True)})")

    autores_str = "; ".join(autores)

    # Abstract
    abstract = None
    abstract_div = soup.select_one('div.item.abstract')

    if abstract_div:
        # Remove o rótulo "Resumo", se existir
        label = abstract_div.select_one('h3.label')
        if label:
            label.decompose()

        abstract = abstract_div.get_text(separator=" ", strip=True)

    # Armazenamento do artigo
    dados_artigos.append({
        "edicao": edicao,
        "categoria": categoria,
        "titulo": titulo,
        "autores": autores_str,
        "abstract": abstract,
        "url": paper
    })

# Criação do DataFrame
df = pd.DataFrame(dados_artigos)

# Exportação para CSV
df.to_csv("artigos_wit.csv", index=False, encoding="utf-8-sig")

print("CSV gerado com sucesso: artigos_wit.csv")


Conexão bem sucedida com o paper: https://sol.sbc.org.br/index.php/wit/article/view/35966
Conexão bem sucedida com o paper: https://sol.sbc.org.br/index.php/wit/article/view/35967
Conexão bem sucedida com o paper: https://sol.sbc.org.br/index.php/wit/article/view/35968
Conexão bem sucedida com o paper: https://sol.sbc.org.br/index.php/wit/article/view/35969
Conexão bem sucedida com o paper: https://sol.sbc.org.br/index.php/wit/article/view/35970
Conexão bem sucedida com o paper: https://sol.sbc.org.br/index.php/wit/article/view/35971
Conexão bem sucedida com o paper: https://sol.sbc.org.br/index.php/wit/article/view/35972
Conexão bem sucedida com o paper: https://sol.sbc.org.br/index.php/wit/article/view/35973
Conexão bem sucedida com o paper: https://sol.sbc.org.br/index.php/wit/article/view/35974
Conexão bem sucedida com o paper: https://sol.sbc.org.br/index.php/wit/article/view/35975
Conexão bem sucedida com o paper: https://sol.sbc.org.br/index.php/wit/article/view/35976
Conexão be